In [1]:
#导入所需的库
import os
import math

import torch
import torch.nn as nn
# hugging face的分词器，github地址：https://github.com/huggingface/tokenizers
from tokenizers import Tokenizer
# 用于构建词典
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn.functional import pad, log_softmax
from pathlib import Path
from tqdm import tqdm

In [2]:
# 工作目录、为Transformer放数据、缓存文件的目录
work_dir = Path("./dataset")

In [3]:
# 训练好的模型会放在该目录下，注意隔一段时间就要对模型进行保存，这是深度学习训练的基本
model_dir = Path("./model")

In [4]:
# 上次运行到的地方，如果是第一次运行，为None，如果中途暂停了，下次运行时，指定目前最新的模型即可。
model_checkpoint = None # 'model_10000.pt'

In [5]:
#如果工作目录不存在，则创建一个
if not os.path.exists(work_dir):
    os.makedirs(work_dir)

In [6]:
# 如果模型目录不存在，则创建一个
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [7]:
# 英文句子的文件路径/数据集路径，
en_filepath = './dataset/train.en'
# 中文句子的文件路径/数据集路径
zh_filepath = './dataset/train.zh'

In [8]:
with open(en_filepath, 'r', encoding='utf-8') as file:
    #读取所有行
    english_sentences = file.readlines()
    print("English sentences (first 5 lines):")
    for line in english_sentences[:5]:
        print(line.strip())

English sentences (first 5 lines):
A pair of red - crowned cranes have staked out their nesting territory
A pair of crows had come to nest on our roof as if they had come for Lhamo.
A couple of boys driving around in daddy's car.
A pair of nines? You pushed in with a pair of nines?
Fighting two against one is never ideal,


In [9]:
with open(zh_filepath, 'r', encoding='utf-8') as file:
    #读取所有行
    chinese_sentences = file.readlines()
    print("\nChinese sentences (first 5 lines):")
    for line in chinese_sentences[:5]:
        print(line.strip())


Chinese sentences (first 5 lines):
一对丹顶鹤正监视着它们的筑巢领地
一对乌鸦飞到我们屋顶上的巢里，它们好像专门为拉木而来的。
一对乖乖仔开着老爸的车子。
一对九？一对九你就全下注了？
一对二总不是好事，


In [10]:
#定义一个获取文件行数的办法，在开始训练之前，一定要保证离开两种语言对应的句子是一一对应的
def get_row_count(filepath):
    count = 0
    for _ in open(filepath, encoding='utf-8'):
        count += 1
    return count
en_row_count = get_row_count(en_filepath)
zh_row_count  = get_row_count(zh_filepath)

In [11]:
zh_row_count 

10000000

In [12]:
en_row_count

10000000

In [13]:
en_row_count == zh_row_count

True

In [14]:
# 句子数量，主要用于后面显示进度。
row_count = en_row_count

assert  en_row_count != zh_row_count #Assert是一种让代码主动发起报错的经典Python用法

In [15]:
import random

def sample_file_statistics(filepath, sample_fraction=0.01):
    # 初始化统计变量
    total_length = 0
    length_square_sum = 0
    max_length = 0
    count = 0
    
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            # 计算行长度
            length = len(line.strip())
            # 累加总长度
            total_length += length
            # 累加长度的平方，用于方差计算
            length_square_sum += length ** 2
            # 更新最大长度
            max_length = max(max_length, length)
            # 计数
            count += 1

    # 计算平均长度和方差
    if count > 0:
        average_length = total_length / count
        variance = (length_square_sum - (total_length ** 2) / count) / count
    else:
        average_length = 0
        variance = 0

    return average_length, max_length, variance

# 抽样并计算英文和中文句子的平均长度、最大长度和方差
en_stats = sample_file_statistics(en_filepath)
zh_stats = sample_file_statistics(zh_filepath)

print(f"英文句子 平均长度 = {en_stats[0]:.2f}, 最大长度 = {en_stats[1]}, 方差 = {en_stats[2]:.2f}")
print(f"中文句子: 平均长度 = {zh_stats[0]:.2f}, 最大长度 = {zh_stats[1]}, 方差 = {zh_stats[2]:.2f}")

英文句子 平均长度 = 54.21, 最大长度 = 1039, 方差 = 1717.04
中文句子: 平均长度 = 17.17, 最大长度 = 519, 方差 = 162.83


In [16]:
#定义句子最大长度，如果句子不够长，填充，如果超出裁剪
max_length = 72 #方差比较大，设置长一点
print("句子数量为：", en_row_count)
print("句子最大长度为：", max_length)

句子数量为： 10000000
句子最大长度为： 72


In [17]:
#定义中英文词典
en_vocab = None
zh_vocab = None


In [18]:
#定义bathc_size, 由于是文本，占用内存小，可以大一点
batch_size = 128

In [19]:
#epochs 不用设置太大，句子越多，需要的训练次数影响越小
epochs = 1

In [20]:
save_after_step = 5000 #多少小步保存一次模型，防止程序崩溃导致模型丢失


In [21]:
#是否使用缓存,由于文件较大，初始化松佐较慢，所以初始化好的文件需要持久化
use_cache = True

In [24]:
#定义训练设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [23]:
device

device(type='cpu')

In [25]:
print("batch_size:", batch_size)
print("每{}步保存一次模型".format(save_after_step))
print("Device:", device)

batch_size: 128
每5000步保存一次模型
Device: cpu


!ping huggingface.co 

In [26]:
#加载基础分词器模型，使用的是基础的bert,`uncased`意思是不区分大小写
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [28]:
# def en_tokenizer(line):
#     """
#     定义英文分词器，后续也要使用
#     :param line: 一句英文句子，例如"I'm learning Deep learning."
#     :return: subword分词后的记过，例如：['i', "'", 'm', 'learning', 'deep', 'learning', '.']
#     """
#     # 使用bert进行分词，并获取tokens。add_special_tokens是指不要在结果中增加‘<bos>’和`<eos>`等特殊字符
#     return tokenizer.encode(line)

In [29]:
def en_tokenizer(line):
    return tokenizer.convert_ids_to_tokens(tokenizer.encode(line,  add_special_tokens=False))

In [30]:
print(en_tokenizer("I'm a English tokenizer."))

['i', "'", 'm', 'a', 'english', 'token', '##izer', '.']


In [31]:
def yield_en_tokens():
    """
    每次yield一个分词之后的英文句子，之所以yield是为了节省内存
    如果先分好词语再构建词典，会有大量文本驻留内存，造成内存溢出
    """
    flie = open(en_filepath, encoding='utf-8')
    for line in tqdm(flie,desc="构建英文词典", total=row_count):
        yield en_tokenizer(line)
    flie.close()

In [32]:
#指定英文词典缓存路径
en_vocab_file = work_dir / "vocab_en.pt"
#如果使用缓存，且文件存在，加载缓存文件
if use_cache and os.path.exists(en_vocab_file):
    en_vocab = torch.load(en_vocab_file, map_location="cpu")
else:
    en_vocab = build_vocab_from_iterator(
        # 传入一个形如[['i', 'am', ...], ['machine', 'learning', ...], ...]的可迭代列表
        #这里可以理解直接传入了一个迭代器
        yield_en_tokens(),
        #最小频率为2，即一个单词最少收录两次才会被收录入到词典
        min_freq = 2,
        #在词典最开始加上这些特殊token
        specials = ["<s>", "</s>", "<pad>", "<unk>"],
    )
    #设置词典默认的index,后面文本转index时，如果找不到,就会用这个index
    en_vocab.set_default_index(en_vocab["<unk>"])
    #保存缓存文件
    if use_cache:
        torch.save(en_vocab, en_vocab_file)
        

In [33]:
#查查查看一下词典的前10个词
print("英文词典大小：", len(en_vocab))
print(dict((i, en_vocab.lookup_token(i)) for i in range(10)))


英文词典大小： 27584
{0: '<s>', 1: '</s>', 2: '<pad>', 3: '<unk>', 4: '.', 5: ',', 6: 'the', 7: "'", 8: 'i', 9: 'you'}


构造中文词典

In [34]:
def zh_tokenizer(line):
    """
    定义中文分词器
    :param line: 中文句子，例如：机器学习
    :return: 分词结果，例如['机','器','学','习']
    """
    #strip 删除头尾指定字符，默认是空格
    return list(line.strip().replace(" ", ""))


def yield_zh_tokens():
    file = open(zh_filepath, encoding='utf-8')
    for line in tqdm(file, desc="构建中文词典", total=row_count):
        yield zh_tokenizer(line)
    file.close()

In [35]:
zh_vocab_file = work_dir / "vocab_zh.pt"
if use_cache and os.path.exists(zh_vocab_file):
    zh_vocab = torch.load(zh_vocab_file, map_location="cpu")
else:
    zh_vocab = build_vocab_from_iterator(
        yield_zh_tokens(),
        min_freq=1,
        specials=["<s>", "</s>", "<pad>", "<unk>"],
    )
    zh_vocab.set_default_index(zh_vocab["<unk>"])
    torch.save(zh_vocab, zh_vocab_file)

In [36]:
# 打印看一下效果，用前10个词为例
print("中文词典大小:", len(zh_vocab))
print(dict((i, zh_vocab.lookup_token(i)) for i in range(10)))

中文词典大小: 8280
{0: '<s>', 1: '</s>', 2: '<pad>', 3: '<unk>', 4: '。', 5: '的', 6: '，', 7: '我', 8: '你', 9: '是'}


In [37]:

zh_vocab( zh_tokenizer("一对二总不是好事"))

[12, 40, 516, 281, 11, 9, 27, 64]

数据批量加载为tensor

In [38]:
class TranslationDataset(Dataset):

    def __init__(self):
        # 加载英文tokens
        self.en_tokens = self.load_tokens(en_filepath, en_tokenizer, en_vocab, "构建英文tokens", 'en')
        # 加载中文tokens
        self.zh_tokens = self.load_tokens(zh_filepath, zh_tokenizer, zh_vocab, "构建中文tokens", 'zh')

    def __getitem__(self, index):
        return self.en_tokens[index], self.zh_tokens[index]

    def __len__(self):
        return row_count

    def load_tokens(self, file, tokenizer, vocab, desc, lang):
        """
        加载tokens，即将文本句子们转换成index们。
        :param file: 文件路径，例如"./dataset/train.en"
        :param tokenizer: 分词器，例如en_tokenizer函数
        :param vocab: 词典, Vocab类对象。例如 en_vocab
        :param desc: 用于进度显示的描述，例如：构建英文tokens
        :param lang: 语言。用于构造缓存文件时进行区分。例如：’en‘
        :return: 返回构造好的tokens。例如：[[6, 8, 93, 12, ..], [62, 891, ...], ...]
        """

        # 定义缓存文件存储路径
        cache_file = work_dir / "tokens_list.{}.pt".format(lang)
        # 如果使用缓存，且缓存文件存在，则直接加载
        if use_cache and os.path.exists(cache_file):
            print(f"正在加载缓存文件{cache_file}, 请稍后...")
            return torch.load(cache_file, map_location="cpu")

        # 从0开始构建，定义tokens_list用于存储结果
        tokens_list = []
        # 打开文件
        with open(file, encoding='utf-8') as file:
            # 逐行读取
            for line in tqdm(file, desc=desc, total=row_count):
                # 进行分词
                tokens = tokenizer(line)
                # 将文本分词结果通过词典转成index
                tokens = vocab(tokens)
                # append到结果中
                tokens_list.append(tokens)
        # 保存缓存文件
        if use_cache:
            torch.save(tokens_list, cache_file)

        return tokens_list

In [39]:
dataset = TranslationDataset()

正在加载缓存文件dataset\tokens_list.en.pt, 请稍后...
正在加载缓存文件dataset\tokens_list.zh.pt, 请稍后...


In [42]:
print(dataset[0])

([11, 2730, 12, 554, 19, 17210, 18077, 27, 3078, 203, 57, 102, 18832, 3653], [12, 40, 1173, 1084, 3169, 164, 693, 397, 84, 100, 14, 5, 1218, 2397, 535, 67])


填充与裁剪句子，使句子等长

In [43]:
def collate_fn(batch):
    """
    将dataset的数据进一步处理，并组成一个batch。
    :param batch: 一个batch的数据，例如：
                  [([6, 8, 93, 12, ..], [62, 891, ...]),
                  ....
                  ...]
    :return: 填充后的且等长的数据，包括src, tgt, tgt_y, n_tokens
             其中src为原句子，即要被翻译的句子
             tgt为目标句子：翻译后的句子，但不包含最后一个token
             tgt_y为label：翻译后的句子，但不包含第一个token，即<bos>
             n_tokens：tgt_y中的token数，<pad>不计算在内。
    """

    # 定义'<bos>'的index，在词典中为0，所以这里也是0
    bs_id = torch.tensor([0])
    # 定义'<eos>'的index
    eos_id = torch.tensor([1])
    # 定义<pad>的index
    pad_id = 2

    # 用于存储处理后的src和tgt
    src_list, tgt_list = [], []

    # 循环遍历句子对儿
    for (_src, _tgt) in batch:
        """
        _src: 英语句子，例如：`I love you`对应的index
        _tgt: 中文句子，例如：`我 爱 你`对应的index
        """

        processed_src = torch.cat(
            # 将<bos>，句子index和<eos>拼到一块
            [
                bs_id,
                torch.tensor(
                    _src,
                    dtype=torch.int64,
                ),
                eos_id,
            ],
            0,
        )
        processed_tgt = torch.cat(
            [
                bs_id,
                torch.tensor(
                    _tgt,
                    dtype=torch.int64,
                ),
                eos_id,
            ],
            0,
        )

        """
        将长度不足的句子进行填充到max_padding的长度的，然后增添到list中

        pad：假设processed_src为[0, 1136, 2468, 1349, 1]
             第二个参数为: (0, 72-5)
             第三个参数为：2
        则pad的意思表示，给processed_src左边填充0个2，右边填充67个2。
        最终结果为：[0, 1136, 2468, 1349, 1, 2, 2, 2, ..., 2]
        """
        src_list.append(
            pad(
                processed_src,
                (0, max_length - len(processed_src),),
                value=pad_id,
            )
        )
        tgt_list.append(
            pad(
                processed_tgt,
                (0, max_length - len(processed_tgt),),
                value=pad_id,
            )
        )

    # 将多个src句子堆叠到一起
    src = torch.stack(src_list)
    tgt = torch.stack(tgt_list)

    # tgt_y是目标句子去掉第一个token，即去掉<bos>
    tgt_y = tgt[:, 1:]
    # tgt是目标句子去掉最后一个token
    tgt = tgt[:, :-1]

    # 计算本次batch要预测的token数
    n_tokens = (tgt_y != 2).sum()

    # 返回batch后的结果
    return src, tgt, tgt_y, n_tokens

In [44]:
torch.cat([torch.tensor([0]),torch.tensor([0])], 0).sum()

tensor(0)

In [45]:
#dataLoader的本质是从dataset中取出一个batch的数据后，将一整个batch的数据放入函数collate_fn中处理后返回
#默认的collate_fn函数是
'''
collate_fn=lambda x:(
 torch.cat(
  [x[i][j].unsqueeze(0) for i in range(len(x))], 0
  ) for j in range(len(x[0]))
 )
'''
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [46]:
src, tgt, tgt_y, n_tokens = next(iter(train_loader))
src, tgt, tgt_y = src.to(device), tgt.to(device), tgt_y.to(device)

In [47]:
print("src.size:", src.size())
print("tgt.size:", tgt.size())
print("tgt_y.size:", tgt_y.size())
print("n_tokens:", n_tokens)

src.size: torch.Size([128, 72])
tgt.size: torch.Size([128, 71])
tgt_y.size: torch.Size([128, 71])
n_tokens: tensor(2410)


In [48]:
class PositionalEncoding(nn.Module):
    "进行位置编码."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 初始化Shape为(max_len, d_model)的PE (positional encoding)
        pe = torch.zeros(max_len, d_model).to(device)
        # 初始化一个tensor [[0, 1, 2, 3, ...]]
        position = torch.arange(0, max_len).unsqueeze(1)
        # 这里就是sin和cos括号中的内容，通过e和ln进行了变换
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        # 计算PE(pos, 2i)
        pe[:, 0::2] = torch.sin(position * div_term)
        # 计算PE(pos, 2i+1)
        pe[:, 1::2] = torch.cos(position * div_term)
        # 为了方便计算，在最外面在unsqueeze出一个batch
        pe = pe.unsqueeze(0)
        # 如果一个参数不参与梯度下降，但又希望保存model的时候将其保存下来
        # 这个时候就可以用register_buffer
        self.register_buffer("pe", pe)

    def forward(self, x):
        """
        x 为embedding后的inputs，例如(1,7, 128)，batch size为1,7个单词，单词维度为128
        """
        # 将x和positional encoding相加。
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

Embedding与掩码以及模型构建

In [49]:
class TranslationModel(nn.Module):

    def __init__(self, d_model, src_vocab, tgt_vocab, dropout=0.1):
        super(TranslationModel, self).__init__()

        # 定义原句子的embedding
        # embedding的维度被设置成了超参数
        # 在本次案例中我们使用的是256
        self.src_embedding = nn.Embedding(len(src_vocab), d_model, padding_idx=2)
        # 定义目标句子的embedding
        self.tgt_embedding = nn.Embedding(len(tgt_vocab), d_model, padding_idx=2)
        # 定义posintional encoding
        self.positional_encoding = PositionalEncoding(d_model, dropout, max_len=max_length)
        # 定义Transformer
        self.transformer = nn.Transformer(d_model, dropout=dropout, batch_first=True)

        # 定义最后的预测层，这里并没有定义Softmax，而是把他放在了模型外。
        self.predictor = nn.Linear(d_model, len(tgt_vocab))

    def forward(self, src, tgt):
        """
        进行前向传递，输出为Decoder的输出。注意，这里并没有使用self.predictor进行预测，
        因为训练和推理行为不太一样，所以放在了模型外面。
        :param src: 原batch后的句子，例如[[0, 12, 34, .., 1, 2, 2, ...], ...]
        :param tgt: 目标batch后的句子，例如[[0, 74, 56, .., 1, 2, 2, ...], ...]
        :return: Transformer的输出，或者说是TransformerDecoder的输出。
        """

        """
        生成tgt_mask，即阶梯型的mask，例如：
        [[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]]
        tgt.size()[-1]为目标句子的长度。
        """
        # 对目标句子，要掩盖住embedding矩阵的上半部分（代表从过去向未来询问的部分）
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt.size()[-1]).to(device)
        # 但除此之外，我们还需要对所有填充的部分进行掩码，减少对模型的噪音干扰
        # 掩盖住原句子中<pad>的部分，例如[[False,False,False,..., True,True,...], ...]
        src_key_padding_mask = TranslationModel.get_key_padding_mask(src)
        # 掩盖住目标句子中<pad>的部分
        tgt_key_padding_mask = TranslationModel.get_key_padding_mask(tgt)

        # 对src和tgt进行编码
        src = self.src_embedding(src)
        tgt = self.tgt_embedding(tgt)
        # 给src和tgt的token增加位置信息
        src = self.positional_encoding(src)
        tgt = self.positional_encoding(tgt)
        # 经过这一步之后，数据的结构变成了三维
        #(batch_size, sentence_len, embedding_dimension)

        # 将准备好的数据送给nn.transformer
        # 在这里我们是一并将源数据与目标数据给到了transformer模型
        # 但transformer模型实际上是先试用src原始数据
        # 再试用tgt目标数据的
        # 同时两个数据的掩码也各不相同
        out = self.transformer(src, tgt,
                               tgt_mask=tgt_mask,
                               src_key_padding_mask=src_key_padding_mask,
                               tgt_key_padding_mask=tgt_key_padding_mask)

        """
        这里直接返回transformer的结果。因为训练和推理时的行为不一样，
        所以在该模型外再进行线性层的预测。
        """
        return out

    @staticmethod
    def get_key_padding_mask(tokens):
        """
        用于key_padding_mask
        """
        return tokens == 2

In [50]:
if model_checkpoint:
    model = torch.load(model_dir / model_checkpoint)
else:
    model = TranslationModel(256, en_vocab, zh_vocab)
model = model.to(device)

In [51]:
model(src, tgt).size()

c:\Users\15913\anaconda3\envs\pytorchcpu\Lib\site-packages\torch\nn\functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


torch.Size([128, 71, 256])

In [52]:
for i in model.parameters():
    print(i)
    break

Parameter containing:
tensor([[-0.8971,  3.4824, -0.6170,  ...,  1.1734,  0.2916,  0.2956],
        [-1.1413, -1.2990,  1.1521,  ...,  0.8747,  0.4199, -1.3246],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.0337, -0.4230, -1.1757,  ..., -0.5287, -0.0297, -0.8514],
        [ 0.9629, -0.6134, -1.0458,  ...,  1.0507,  0.2567, -0.5436],
        [ 0.9033,  1.7036,  0.5967,  ..., -0.3990, -1.4044,  0.6190]],
       requires_grad=True)


In [53]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [54]:
t = torch.tensor([[1,2],[3,4]])
torch.gather(t, 1, torch.tensor([[0,0],[1,0]]))

tensor([[1, 1],
        [4, 3]])

In [60]:
class TranslationLoss(nn.Module):

    def __init__(self):
        super(TranslationLoss, self).__init__()
        # 使用KLDivLoss，不需要知道里面的具体细节。
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = 2

    def forward(self, x, target):
        """
        损失函数的前向传递
        :param x: 将Decoder的输出再经过predictor线性层之后的输出。
                  也就是Linear后、Softmax前的状态
        :param target: tgt_y。也就是label，例如[[1, 34, 15, ...], ...]
        :return: loss
        """

        """
        由于KLDivLoss的input需要对softmax做log，所以使用log_softmax。
        等价于：log(softmax(x))
        """
        x = log_softmax(x, dim=-1)

        """
        构造Label的分布，也就是将[[1, 34, 15, ...]] 转化为:
        [[[0, 1, 0, ..., 0],
          [0, ..., 1, ..,0],
          ...]],
        ...]
        """
        # 首先按照x的Shape构造出一个全是0的Tensor
        true_dist = torch.zeros(x.size()).to(device)
        # 将对应index的部分填充为1
        true_dist.scatter_(1, target.data.unsqueeze(1), 1)
        # 找出<pad>部分，对于<pad>标签，全部填充为0，没有1，避免其参与损失计算。
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)

        # 计算损失
        return self.criterion(x, true_dist.clone().detach())

In [56]:
criteria = TranslationLoss()

In [57]:
writer = SummaryWriter(log_dir='runs/transformer_loss')

In [54]:
torch.cuda.empty_cache()

In [62]:
device

device(type='cpu')

In [56]:
# step = 0

# if model_checkpoint:
#     step = int('model_10000.pt'.replace("model_", "").replace(".pt", ""))

# model.train()
# for epoch in range(epochs):
#     loop = tqdm(enumerate(train_loader), total=len(train_loader))
#     for index, data in enumerate(train_loader):
#         # 生成数据
#         src, tgt, tgt_y, n_tokens = data
#         src, tgt, tgt_y = src.to(device), tgt.to(device), tgt_y.to(device)

#         # 清空梯度
#         optimizer.zero_grad()
#         # 进行transformer的计算
#         out = model(src, tgt)
#         # 将结果送给最后的线性层进行预测
#         out = model.predictor(out)

#         """
#         计算损失。由于训练时我们的是对所有的输出都进行预测，所以需要对out进行reshape一下。
#                 我们的out的Shape为(batch_size, 词数, 词典大小)，view之后变为：
#                 (batch_size*词数, 词典大小)。
#                 而在这些预测结果中，我们只需要对非<pad>部分进行，所以需要进行正则化。也就是
#                 除以n_tokens。
#         """
#         loss = criteria(out.contiguous().view(-1, out.size(-1)), tgt_y.contiguous().view(-1)) / n_tokens
#         # 计算梯度
#         loss.backward()
#         # 更新参数
#         optimizer.step()

#         loop.set_description("Epoch {}/{}".format(epoch, epochs))
#         loop.set_postfix(loss=loss.item())
#         loop.update(1)

#         step += 1

#         del src
#         del tgt
#         del tgt_y

#         if step != 0 and step % save_after_step == 0:
#             torch.save(model, model_dir / f"model_{step}.pt")

In [64]:
path = Path(r"./model./model_75000.pt")
model = torch.load(path,   map_location="cpu")

In [65]:
#设置为测试模式
model = model.eval()

In [66]:
def translate(src: str):
    """
    :param src: 英文句子，例如 "I like machine learning."
    :return: 翻译后的句子，例如：”我喜欢机器学习“
    """

    # 将与原句子分词后，通过词典转为index，然后增加<bos>和<eos>
    src = torch.tensor([0] + en_vocab(en_tokenizer(src)) + [1]).unsqueeze(0).to(device)
    # 首次tgt为<bos>
    tgt = torch.tensor([[0]]).to(device)
    # 一个一个词预测，直到预测为<eos>，或者达到句子最大长度
    for i in range(max_length):
        # 进行transformer计算
        out = model(src, tgt)
        # 预测结果，因为只需要看最后一个词，所以取`out[:, -1]`
        predict = model.predictor(out[:, -1])
        # 找出最大值的index
        y = torch.argmax(predict, dim=1)
        # 和之前的预测结果拼接到一起
        tgt = torch.concat([tgt, y.unsqueeze(0)], dim=1)
        # 如果为<eos>，说明预测结束，跳出循环
        if y == 1:
            break
    # 将预测tokens拼起来
    tgt = ''.join(zh_vocab.lookup_tokens(tgt.squeeze().tolist())).replace("<s>", "").replace("</s>", "")
    return tgt

In [67]:
translate("What can I say? Wish you a nice day!")

c:\Users\15913\anaconda3\envs\pytorchcpu\Lib\site-packages\torch\nn\functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


'我能说什什什？我能说什什么？祝你一天好好日祝你一天气祝你一天气祝你一天好好美好的一天！祝你一天！祝你一天！祝你一天！美美美美美美美美美美美美美美'

In [68]:
#英译中测试
translate("Alright, this project is finished. Let's see how good this is.")

'好，这个项完成了。我们看看这个项目有多好。'

In [69]:
#
translate("Long.")

'很长长长长长。'

In [70]:
#
translate("I like machine learning.")

'我喜喜喜喜喜喜喜喜喜喜喜喜喜喜喜机机机器学习机器学习机器学习。学习机器学习。'

In [70]:
translate("How are you?")

'你怎你怎怎你怎怎怎怎你怎怎怎怎你怎样如怎你怎怎怎样？你怎怎你怎样如怎？你怎怎？你怎怎怎？你怎怎？你怎？你怎？你怎？你怎怎？你怎？你怎？你怎怎？你'

In [77]:
translate("i have a cat.")

c:\Users\15913\anaconda3\envs\pytorchcpu\Lib\site-packages\torch\nn\functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


'我有只猫猫猫猫猫猫猫猫我有猫猫猫猫猫。'